# Building an Agent Reasoning Loop

## Setup

In [1]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

import nest_asyncio
nest_asyncio.apply()

## Load the data

## Setup the Query Tools

In [2]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [3]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [4]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [5]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has distinct responsibilities and contributes to the software development process in a structured manner. The Product Manager focuses on requirements and user stories, the Architect designs the system architecture, the Project Manager allocates tasks, the Engineer implements the code, and the QA Engineer ensures quality through testing. These roles collaborate in a workflow to achieve successful software development outcomes within the MetaGPT framework.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Agents i

In [6]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-10-16
last_modified_date: 2024-10-16

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

In [7]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset. The HumanEval dataset consists of 164 handwritten programming tasks, while the MBPP dataset consists of 427 Python tasks. The SoftwareDev dataset comprises 70 representative software development tasks covering various scopes such as mini-games, image processing algorithms, and data visualization.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset. HumanEval consists of 164 handwritten programming tasks, MBPP includes 427 Python tasks, and the SoftwareDev dataset comprises 70 representative software development tasks covering various scopes.


In [8]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over the HumanEval dataset", "page_numbers": ["7"]}
=== Function Output ===
MetaGPT achieved 85.9% and 87.7% Pass rates over the HumanEval dataset.
=== LLM Response ===
MetaGPT achieved pass rates of 85.9% and 87.7% over the HumanEval dataset.


## Lower-Level: Debuggability and Control

In [9]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [10]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [11]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has distinct responsibilities and contributes uniquely to the software development process within the MetaGPT system.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Agents in MetaGPT communicate with each other through structured communication interfaces, utilizing a shared message pool for exchanging messages directly. They publish structured messages in the pool and access messages from other agents transparently. Additionally, agents can subscribe to relevant information based on their role profiles, e

In [12]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 964f809b-44b2-4fef-94fe-e7d53e450ffa: 1
The agent roles in MetaGPT include the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has distinct responsibilities and contributes uniquely to the software development process within the MetaGPT system.


In [13]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 964f809b-44b2-4fef-94fe-e7d53e450ffa: 1


TaskStep(task_id='964f809b-44b2-4fef-94fe-e7d53e450ffa', step_id='946dde60-7323-41f8-b1d1-61863b581f1b', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [14]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT share information through a shared message pool and utilize a subscription mechanism to manage information overload. This allows them to publish structured messages and subscribe to relevant messages based on their profiles, facilitating efficient communication within the multi-agent framework. The global message pool ensures streamlined communication by promoting efficiency and relevance in the information exchanged, while the subscription mechanism filters out irrelevant contexts, enhancing the utility and relevance of the shared information.


In [15]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
Agents in MetaGPT share information through a shared message pool and utilize a subscription mechanism to manage information overload. This allows them to publish structured messages and subscribe to relevant messages based on their profiles, facilitating efficient communication within the multi-agent framework. The global message pool ensures streamlined communication by promoting efficiency and relevance in the information exchanged, while the subscription mechanism filters out irrelevant contexts, enhancing the utility and relevance of the shared information.
True


In [16]:
response = agent.finalize_response(task.task_id)

In [17]:
print(str(response))

Agents in MetaGPT share information through a shared message pool and utilize a subscription mechanism to manage information overload. This allows them to publish structured messages and subscribe to relevant messages based on their profiles, facilitating efficient communication within the multi-agent framework. The global message pool ensures streamlined communication by promoting efficiency and relevance in the information exchanged, while the subscription mechanism filters out irrelevant contexts, enhancing the utility and relevance of the shared information.
